In [5]:
#Imports
import pandas as pd
import numpy as np
import joblib


In [48]:
#importing 1st transactions
test_df_sample= pd.read_csv("test_sample.csv")
ids = test_df_sample['msno'].copy()
test_df_sample.drop(['msno', 'is_churn', 'bd','payment_method_id', 'city', 'registered_via'], axis=1, inplace=True)


In [43]:
#import model and preprocessor

package = joblib.load("../Churn_prediction/model.pkl")

loaded_model = package.named_steps['classifier']
loaded_preproc = package.named_steps['preprocessor']

In [53]:
test_df_sample_preproc = loaded_preproc.fit_transform(test_df_sample)


In [45]:
#list features for their scalers
robust_features = [               'remaining_plan_duration', 
          'usage_from_ltd',
              'payment_plan_days',
              'plan_list_price',
              'actual_amount_paid',
             'num_50',
              'num_75',
              'num_985',
              
              'expire_year',
              'last_transaction_year'
             ]
minmax_features = ['registration_year']
normal_features = [ 'num_25',
              
              'num_100',
              'num_unq',
              'total_secs',]

transformed_columns = robust_features + minmax_features + normal_features
passthrough_cols = [col for col in test_df_sample.columns if col not in transformed_columns]
new_cols_order = transformed_columns + passthrough_cols

# Convert transformed data back to DataFrame to ensure column order is maintained
test_df_sample_preproc = pd.DataFrame(
    test_df_sample_preproc, 
    columns=new_cols_order
)

In [54]:
predict = loaded_model.predict_proba(test_df_sample_preproc)

In [55]:
new = pd.DataFrame({'ID': ids, 'Prediction percentage': predict[:,1]})
new

,ID,Prediction percentage
0,waapRCaC7ZuXbhspZFVAi6LZlsiGzy1/ZKcx3b3p5k0=,0.001171
1,t8Mm0n+kswYZ1nGCxbHbihjV7LgAIYcuBJ0ijV7MN1c=,0.285515
2,TuIVZ3831MX0E078BnPH14SEvNxo6XZg0lTT5lXe9V4=,0.269922
3,rdqumWKqK+2C+69cqarqFbCFJWGDoWf/oD4ppREUEOc=,0.015125
4,wWpC7gW4v7/L+dAG8O0ggc6gWm1CiVrfdUUE2Q7BYsY=,0.143332
...,...,...
49995,li65iBvJ3V/T8he8cZarGOHtDghyquGRaDh05emy/u8=,0.059956
49996,/xvBUSoWkJpsDyIIgshYXKHOR+b30Lg33OGcZ9S6xTA=,0.001100
49997,amrpjonRx5wr9XFjUjl/YmM3fSCgdZHQJmKE3rmF49g=,0.011415
49998,D2c882fWyJrND6YQ9zf+qsz5f2Uv2PGqNXyH9uM9lLI=,0.028760
